Импортируем модули, которые нам понадобятся для решения задачи

In [36]:
pip install datasketch

In [37]:
pip install python-Levenshtein

In [38]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib
import datasketch
from datasketch import MinHash, MinHashLSH
import Levenshtein

Загрузим наш датасет и выведем первые 4 значения

In [39]:
# loading the data from the csv file to apandas dataframe
movies_data = pd.read_csv('/content/movies_metadata_new 2.csv')

In [40]:
# printing the first 4 rows of the dataframe
movies_data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,30-10-1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,15-12-1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,22-12-1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,22-12-1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,10-02-1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [41]:
# number of rows and columns in the datяa frame

movies_data.shape

(45463, 24)

In [42]:
# selecting the relevant features for recommendation

selected_features = ['genres', 'vote_average', 'popularity', 'production_companies', 'release_date', 'vote_average', 'imdb_id']
print(selected_features)

['genres', 'vote_average', 'popularity', 'production_companies', 'release_date', 'vote_average', 'imdb_id']


In [48]:
# replacing the null valuess with null string

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna(' ')

In [49]:

# combining all the 5 selected features

new_var = movies_data['genres'] + ' ' + movies_data['vote_average'].astype(str)
combined_features = new_var + ' ' + movies_data['tagline'] + ' ' + movies_data['popularity'].astype(str) + ' ' + movies_data['overview']


In [50]:
print(combined_features)

0                                                      NaN
1        [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2        [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3        [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4        [{'id': 35, 'name': 'Comedy'}] 5.7 Just When H...
                               ...                        
45458    [{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...
45459                                                  NaN
45460    [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...
45461                                                  NaN
45462                                                  NaN
Length: 45463, dtype: object


In [51]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [53]:
# Убираем пустые строки
combined_features_cleaned = combined_features.replace('', np.nan).fillna('')

# Применяем TF-IDF векторизацию
feature_vectors = vectorizer.fit_transform(combined_features_cleaned)


In [54]:
print(feature_vectors)

  (1, 29873)	0.12170906580722479
  (1, 62130)	0.11311492760318194
  (1, 50380)	0.08274504204403207
  (1, 47978)	0.17708436993080728
  (1, 34514)	0.09093386108779225
  (1, 55567)	0.18944182107147947
  (1, 37349)	0.11888499823977675
  (1, 36569)	0.05702691195664041
  (1, 56331)	0.1165470299206039
  (1, 62269)	0.09716125034851615
  (1, 35628)	0.07493760897918297
  (1, 62423)	0.08519414775727356
  (1, 21707)	0.06757114606284453
  (1, 22847)	0.05644963242993738
  (1, 55759)	0.14899565494433564
  (1, 53328)	0.12572056883303187
  (1, 66154)	0.08000538712903649
  (1, 35651)	0.13135636691487584
  (1, 41889)	0.043294189852227036
  (1, 36436)	0.11448042028543767
  (1, 39981)	0.105913688748558
  (1, 50081)	0.0735902293567344
  (1, 56063)	0.11529711422744618
  (1, 44983)	0.09251741756308567
  (1, 62254)	0.05842330995519342
  :	:
  (45460, 36660)	0.18743669470507213
  (45460, 53121)	0.20397287698236763
  (45460, 47464)	0.18956131092885067
  (45460, 22925)	0.2154942837411319
  (45460, 25453)	0.185131

Cosine Similarity

In [55]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)


In [56]:
print(similarity.shape)

(45463, 45463)


Дальше нужно будет ввести название любимого фильма, чтобы программа, впоследствие вывела рекомендуемые фильмы


In [57]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : catch me if you can


In [58]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Toy Story', 'Jumanji', 'Grumpier Old Men', 'Waiting to Exhale', 'Father of the Bride Part II', 'Heat', 'Sabrina', 'Tom and Huck', 'Sudden Death', 'GoldenEye', 'The American President', 'Dracula: Dead and Loving It', 'Balto', 'Nixon', 'Cutthroat Island', 'Casino', 'Sense and Sensibility', 'Four Rooms', 'Ace Ventura: When Nature Calls', 'Money Train', 'Get Shorty', 'Copycat', 'Assassins', 'Powder', 'Leaving Las Vegas', 'Othello', 'Now and Then', 'Persuasion', 'The City of Lost Children', 'Shanghai Triad', 'Dangerous Minds', 'Twelve Monkeys', 'Wings of Courage', 'Babe', 'Carrington', 'Dead Man Walking', 'Across the Sea of Time', 'It Takes Two', 'Clueless', 'Cry, the Beloved Country', 'Richard III', 'Dead Presidents', 'Restoration', 'Mortal Kombat', 'To Die For', 'How To Make An American Quilt', 'Se7en', 'Pocahontas', 'When Night Is Falling', 'The Usual Suspects', 'Guardian Angel', 'Mighty Aphrodite', 'Lamerica', 'The Big Green', 'Georgia', 'Kids of the Round Table', 'Home for the Holida

In [59]:
import difflib

# Преобразуем все элементы списка в строки
list_of_all_titles = [str(title) for title in list_of_all_titles]

# Ищем похожие названия фильмов
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)


['Catch Me If You Can', 'Catch Me If You Can']


In [60]:
close_match = find_close_match[0]
print(close_match)

Catch Me If You Can


In [61]:
# finding the index of the movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['id'].values[0]
print(index_of_the_movie)

640


In [62]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.0), (1, 0.1024384147840443), (2, 0.08768233755754347), (3, 0.08134623590088241), (4, 0.12233980676917704), (5, 0.06896631720051448), (6, 0.10438089627829658), (7, 0.09203954442085552), (8, 0.0706662366717985), (9, 0.07640990087996305), (10, 0.08845629690822286), (11, 0.0), (12, 0.031209254932365846), (13, 0.10344910261098546), (14, 0.1422217627785189), (15, 0.09025301147359069), (16, 0.07634835058318983), (17, 0.1324815867135352), (18, 0.1234826804632099), (19, 0.052545726824691004), (20, 0.10258291057891993), (21, 0.07694432915564836), (22, 0.14694228370917187), (23, 0.12079026877857219), (24, 0.11818746583205382), (25, 0.09860233873635826), (26, 0.10384548956755787), (27, 0.0), (28, 0.0482116174418847), (29, 0.09611178792797855), (30, 0.11221501121895264), (31, 0.17274529486507287), (32, 0.0), (33, 0.1225371249560945), (34, 0.08939062584701454), (35, 0.0), (36, 0.0), (37, 0.04272609334573355), (38, 0.08259180211805542), (39, 0.0), (40, 0.018515868509756197), (41, 0.13139412241

In [63]:
len(similarity_score)

45463

In [64]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(640, 1.0000000000000004), (45420, 0.32443109201586184), (16227, 0.2607498071326639), (33913, 0.2242920098137902), (41055, 0.2232336362888447), (594, 0.22220902522089322), (3895, 0.22138733206169842), (11090, 0.21801107601459668), (14390, 0.21629022953318594), (17575, 0.21572741347359534), (4881, 0.215208146777427), (16921, 0.2150826206041356), (20821, 0.21422152366236585), (1830, 0.21280952088561517), (27170, 0.21245240015216957), (10676, 0.2116545570534327), (8979, 0.2109660074405764), (31105, 0.21095562679754817), (43357, 0.20890396868067518), (2994, 0.20862319457248196), (37179, 0.20765274507821063), (4407, 0.20725051581499512), (13189, 0.20620229310872654), (17755, 0.2059138017737398), (35357, 0.20581409073915366), (28754, 0.20519414952782228), (24776, 0.20438780133255086), (32834, 0.20408348254538458), (9643, 0.20330479526499556), (10225, 0.20311379666885382), (11048, 0.20233686551589194), (27302, 0.20185336762665598), (8583, 0.20153498319023233), (8457, 0.20145537592625393), (1

In [67]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Moll Flanders
2 . The Fortunes and Misfortunes of Moll Flanders
3 . The Human Experience
4 . The Blue Veil
5 . The Telephone Book
6 . The Wooden Man's Bride
7 . The Gift
8 . The Thief Lord
9 . Storm Over Asia
10 . Twixt
11 . Charlotte Gray
12 . Films to Keep You Awake: The Christmas Tale
13 . American Winter
14 . The Best Years of Our Lives
15 . Project: Shadowchaser
16 . Christmas in Connecticut
17 . Only Yesterday
18 . Morning Star
19 . Tommy's Honour
20 . Places in the Heart
21 . Plan 9
22 . The Man from Snowy River
23 . The Reader
24 . María Candelaria
25 . Homeland (Iraq Year Zero)
26 . The Crown Jewels
27 . Stalingrad
28 . ...ing
29 . The Chase


Movie Recommendation Sytem